In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import spartan.utils.utils as spartan_utils
from imitation_agent.dataset.function_factory import ObservationFunctionFactory, ActionFunctionFactory
from imitation_agent.dataset.imitation_episode_sequence_dataset import ImitationEpisodeSequenceDataset
from imitation_agent.dataset.imitation_episode_dataset import ImitationEpisodeDataset
spartan_source_dir = spartan_utils.getSpartanSourceDir()
logs_config_yaml = os.path.join(spartan_source_dir, "modules/imitation_agent/config/task/move_to_box_0710.yaml")
logs_config = spartan_utils.getDictFromYamlFilename(logs_config_yaml)

imitation_src_dir = os.path.join(spartan_source_dir, "modules/imitation_agent")
data_dir = spartan_utils.get_data_dir()
logs_dir_path = os.path.join(data_dir, "pdc/imitation/move_to_box_0710")

config_yaml = os.path.join(imitation_src_dir, "config", "model", "mlp_stateless_position.yaml")
config = spartan_utils.getDictFromYamlFilename(config_yaml)

obs_function = ObservationFunctionFactory.get_function(config)
action_function = ActionFunctionFactory.action_from_config(config)
dataset = ImitationEpisodeDataset(logs_dir_path, logs_config, config,
                                  action_function=action_function,
                                  observation_function=obs_function)

In [ ]:
from matplotlib  import cm

def get_min_max_rewards(df):
    first = True
    for index, row in df.iterrows():
        if first:
            min_r = row["reward"]
            max_r = row["reward"]
            first = False
            continue
        if row["reward"] > max_r:
            max_r = row["reward"]
        if row["reward"] < min_r:
            min_r = row["reward"]
    return min_r, max_r
        
def get_normed_reward(reward, min_r, max_r):
    return (reward - min_r)/(max_r-min_r)
        
        
def create_distribution_plots(ax, df, dataset, min_r, max_r, first):
    
    xs = []
    ys = []
    cs = []
    
    # plot heatmap
    for index, row in df.iterrows():
        object_position = row["object_position"].strip("[").strip("]").split(",")
        object_position = [float(x) for x in object_position]
        object_position = np.asarray(object_position)
        reward = row["reward"]
        #jet = cm.get_cmap("jet")
        #jet = cm.get_cmap("coolwarm")
        #normed_reward = (-1.0*get_normed_reward(reward, min_r, max_r))+1.0
        #normed_reward = get_normed_reward(reward, min_r, max_r)
        xs.append(object_position[0])
        ys.append(object_position[1])
        #cs.append(jet(normed_reward))
        cs.append(reward)
    [xs, ys, cs] = [np.asarray(x) for x in [xs,ys,cs]]
    
    hm = ax.scatter(xs, ys, c=cs, vmin=min_r, vmax=max_r, cmap = cm.coolwarm_r)
    #hm = ax.scatter(ys, xs, c=cs, cmap = cm.coolwarm_r)
    #plt.colorbar(hm)
            
    # plot training examples
    for log_name in dataset.episodes.keys():
        episode = dataset.episodes[log_name]
        object_pose_trained = episode.sim_config_dict["instances"][0]["q0"]
        sc = ax.scatter(object_pose_trained[0], object_pose_trained[1], c="gray", alpha=0.4, marker="x")
    
    ax.axis('equal')
    ax.set(ylim=(-0.2, 0.2), xlim=(0.6, 0.7))
    ax.set_title(row['name'])
    if not first:
        #ax.set_xticks([])
        ax.set_yticks([])
    else:
        ax.set_ylabel("y, box position")
        ax.set_xlabel("x, box position")
    return hm
    
        
dfs = []  

folders = []
for folder in sorted(os.listdir(os.path.join(spartan_source_dir, "sandbox/experiment_01-aug5"))):
    print folder
    folders.append(folder)
    path_to_move_to_box_folder = os.path.join(spartan_source_dir, "sandbox","experiment_01-aug5",folder)
    path_to_csv = os.path.join(path_to_move_to_box_folder, "results.csv")
    df = pd.read_csv(path_to_csv, index_col=0)
    df["name"] = folder[3:]
    dfs.append(df)
    
min = 1e6
max = -1e6
for df in dfs:
    this_min, this_max = get_min_max_rewards(df)
    if this_min < min:
        min = this_min
    if this_max > max:
        max = this_max
        
# SETTING THIS MANUALLY FOR SCALE
min = -10

fig, axes = plt.subplots(ncols=len(dfs),nrows=1)
fig.set_figheight(5)
fig.set_figwidth(15)
for i, df in enumerate(dfs):
    ax = axes[i]
    if i == 0:
        first = True
    else:
        first = False
    hm = create_distribution_plots(ax, df, dataset, min, max, first=first)
#plt.colorbar(hm, pad=0.2)

cbaxes = fig.add_axes([0.92, 0.12, 0.02, 0.76]) 
cb = plt.colorbar(hm, cax = cbaxes) 
cb.set_label('Task success metric (higher is better)', labelpad=20, rotation=270)

plt.show()

In [ ]:
import numpy as np
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

import matplotlib as mpl
import matplotlib.pyplot as plt

def plot_tensorflow_log(axes, path, color, label):

    event_acc = EventAccumulator(path)#, tf_size_guidance)
    event_acc.Reload()

    # Show all tags in the log file
    #print(event_acc.Tags())

    training_accuracies =   event_acc.Scalars('loss')
    validation_accuracies = event_acc.Scalars('test_loss_average')

    x_train_steps = []
    x_train_values = []
    
    
    for i in xrange(len(training_accuracies)):
        x_train_steps.append(training_accuracies[i][1]) # 1 is step
        x_train_values.append(training_accuracies[i][2]) # 2 is step
    
#     from scipy.signal import savgol_filter
#     x_train_values_filt = savgol_filter(x_train_values, 21, 2)

    #from scipy.signal import medfilt
    #x_train_values_filt = medfilt(x_train_values, 21)
    
    w = 0.90
    x_train_values = np.asarray(x_train_values)
    x_train_values_filt = x_train_values*0.0
    x_train_values_filt[0] = x_train_values[0]
    for i, v in enumerate(x_train_values[1:]):
        x_train_values_filt[i+1] = x_train_values_filt[i]*w + (1-w)*x_train_values[i]
   
    
#     from scipy import signal
#     xn = x_train_values
#     b, a = signal.butter(3, 0.05)
#     zi = signal.lfilter_zi(b, a)
#     z, _ = signal.lfilter(b, a, xn, zi=zi*xn[0])
#     z2, _ = signal.lfilter(b, a, z, zi=zi*z[0])
#     y = signal.filtfilt(b, a, xn)
#     x_train_values_filt = y

        
    x_validation_steps = []
    x_validation_values = []
    
    for i in xrange(len(validation_accuracies)):
        x_validation_steps.append(validation_accuracies[i][1])
        x_validation_values.append(validation_accuracies[i][2])
        

    axes[0].plot(np.asarray(x_train_steps), np.asarray(x_train_values_filt), color=color, label=label, alpha=1.0)
    axes[0].plot(np.asarray(x_train_steps), np.asarray(x_train_values), color=color, alpha=0.1)
    axes[1].plot(np.asarray(x_validation_steps), np.asarray(x_validation_values), label=label, color=color)

    axes[0].set_xlabel("Steps")
    axes[1].set_xlabel("Steps")
    axes[0].set_ylabel("Loss")
    axes[1].set_ylabel("Loss")
    axes[0].set_title("Train")
    axes[0].set(ylim=(0.6e-4, 1.5e-4))
    axes[1].set(ylim=(0.6e-4, 1.5e-4))
    axes[1].set_title("Validation")


def get_tf_events_file(path):
    return sorted(os.listdir(path))[-1]
    
base = os.path.join(data_dir, "pdc/imitation/trained_models/mlp_position/experiment_01-aug5")

fig, axes = plt.subplots(ncols=2,nrows=1)
fig.set_figheight(5)
fig.set_figwidth(15)



#colors = ["black", "red", "orange", "green", "blue", "purple"]
colors = ["red", "orange", "green", "blue", "purple"]

#folders.insert(0,"00-blind")
#folders.remove("00-blind")

counter = 0
for color, folder in zip(colors, folders):
    counter+=1
    path_to = os.path.join(base,folder,"tensorboard")
    tf_file = get_tf_events_file(path_to)
    tf_file_full = os.path.join(path_to, tf_file)
    plot_tensorflow_log(axes, tf_file_full, color, label=folder[3:])
#     if counter == 2:
#         break
    
axes[0].ticklabel_format(style='sci', axis='y', scilimits=(0,0))
axes[1].ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.legend(loc='upper right', frameon=True)
plt.show()


In [ ]:
print "Training times"
print "---------------"
import datetime

for folder in folders:
    path_to = os.path.join(base,folder,"tensorboard")
    tf_file = get_tf_events_file(path_to)
    tf_file_full = os.path.join(path_to, tf_file)
    event_acc = EventAccumulator(tf_file_full)#, tf_size_guidance)
    event_acc.Reload()
    training_accuracies =   event_acc.Scalars('loss')
    seconds = training_accuracies[-1][0] - training_accuracies[0][0]
    #print str(datetime.timedelta(seconds=seconds)) 
    #print '%d hr, %d minutes, %02d seconds' % (seconds / 60 / 60,  seconds/ 60 % 60, seconds % 60)
    print '{:25}'.format(folder[3:]),  '%d hr, %d minutes' % (seconds / 60 / 60,  seconds/ 60 % 60)
    
    


In [ ]:
print folders

In [ ]:
def create_time_drift_plot(dfs):
    for df in dfs:
        plt.scatter(df["index"], df["termination_time"])
    plt.show()
    
    plt.scatter(dfs[-1]["index"], df["termination_time"])
    plt.show()
    
create_time_drift_plot(dfs)

In [ ]:
import matplotlib

def create_violin_plot(dfs):
    df_all = pd.concat(dfs)
    sns.set(style="whitegrid")
    ax = sns.violinplot(x=df_all["name"], y=df_all["reward"])
    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(15, 4)
    plt.show()
    
create_violin_plot(dfs)

In [ ]:
# great example: https://seaborn.pydata.org/examples/distplot_options.html
# full documentation: https://seaborn.pydata.org/generated/seaborn.distplot.html
def create_dist_plot(dfs):
    df_all = pd.concat(dfs)
    
    fig, axes = plt.subplots(1, len(dfs), figsize=(9, 3), sharey=True)
    #axes[0].set_ylim([-20,0])
    sns.set(style="whitegrid")
    for i, df in enumerate(dfs):
        sns.distplot(df["reward"], vertical=True, ax=axes[i], bins=20)
        #ax = sns.violinplot(x=df_all["name"], y=df_all["reward"])
        #fig = matplotlib.pyplot.gcf()
        #fig.set_size_inches(18.5, 10.5)
    plt.show()

    fig, axes = plt.subplots(1, len(dfs), figsize=(9, 3), sharey=True)
    axes[0].set_ylim([-2,-1])
    sns.set(style="whitegrid")
    for i, df in enumerate(dfs):
        sns.distplot(df["reward"], vertical=True, ax=axes[i], bins=20, rug=True)
        #ax = sns.violinplot(x=df_all["name"], y=df_all["reward"])
        #fig = matplotlib.pyplot.gcf()
        #fig.set_size_inches(18.5, 10.5)
    plt.show()

    
    
create_dist_plot(dfs)